## Multiple Linear Regression

Simple linear regression can easily be extended to include multiple features. This is called multiple linear regression:.
A straight-plane fit is a model of the form

$$
\large y = \large \beta + \large \alpha_1 x_1 + \large \alpha_2 x_2 +...+ \large \alpha_n x_n
$$

where:
<br>$\alpha_i$ is commonly known as the *slope* in multiple dimensions that makes up a plane
<br>$\beta$ is commonly known as the *intercept*. 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Code Dictionary
code | description
-----|------------
`read_csv(file)` | Using the Pandas library, create a dataframe for the dataset
`.loc()` | Access a group of rows and columns by label(s) or a boolean array.
`.get_dummies()` | Convert categorical variable into dummy/indicator variables
`.concat()` | Concatenate pandas dataframes along a particular axis.
`statsmodels` | Library for in depth statistical analysis.
`.OLS(y, X)` | Ordinary least squares aka linear regression.
`.RFECV(model, folds)` | Automatic selection of the best number of features.

Let's take an already known dataset

The data itself is extracted from [Inside Airbnb](http://insideairbnb.com) where is possible to make a very nice visual analysis following the hypothesis:: Airbnb claims to be part of the "sharing economy" and disrupting the hotel industry. However, data shows that the majority of Airbnb listings in most cities are entire homes, many of which are rented all year round - disrupting housing and communities.

In [ ]:
dataset = pd.read_csv('airbnb_amsterdam.csv')

In [ ]:
dataset.head()

In [ ]:
dataset.info()

#### What do we want to predict?
Seperate out the target variable and features

In [ ]:
features = ['reviews', 'overall_satisfaction', 'accommodates', 'bedrooms']

In [ ]:
X = dataset[features]
y = dataset.price

In [ ]:
y.mean()

Learning a multiple regression model
Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features: example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

#### What are the Machine Learning steps?

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

$$
\large y = \large \beta + \large \alpha_1 x_1 + \large \alpha_2 x_2 +...+ \large \alpha_n x_n
$$


Then, fit the model to all the features in the data

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

Let's create new airbnb values and predict

In [ ]:
new_appartment = pd.DataFrame({'reviews':[10],
                               'overall_satisfaction':[5],
                               'accommodates':[3],
                               'bedrooms':[1]})

In [ ]:
new_appartment

In [ ]:
new_price = model.predict(new_appartment)
print ("Predicted Airbnb Value: € {}".format(new_price[0]))

#### Exploring more features

Although we often think of multiple regression as including multiple different features, we can also consider transformations of existing features e.g. the log of the accommodates.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [ ]:
from math import log, sqrt

Create the following 4 new features as column in both TEST and TRAIN data:

bedrooms_squared = bedrooms*bedrooms  
log_accommodates = log(accomodaties)

In [ ]:
X_train['bedrooms_squared'] = X_train['bedrooms']*X_train['bedrooms']
X_test['bedrooms_squared'] = X_test['bedrooms']*X_test['bedrooms']
X_train['log_accommodates'] = [log(float(i)) for i in X_train['accommodates']]
X_test['log_accommodates'] = [log(float(i)) for i in X_test['accommodates']]

Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.

#### Features Analysis and Selection
Now we will learn the weights for two (nested) model configuration for predicting prices. The first model will have the fewest features the second model will add the rest.

In [ ]:
model_1_features = ['reviews','overall_satisfaction']
model_2_features = model_1_features + ['accommodates', 'bedrooms','bedrooms_squared','log_accommodates']

Now we train with the new selections

In [ ]:
model_1 = LinearRegression()
model_1.fit(X_train[model_1_features], y_train)

In [ ]:
model_2 = LinearRegression()
model_2.fit(X_train[model_2_features], y_train)

In [ ]:
predictions_1 = model_1.predict(X_test[model_1_features])
predictions_2 = model_2.predict(X_test[model_2_features])

In [ ]:
print ('first real value: {}\nfirst predicted value: {}'.format(y_test.values[0], predictions_1[0]))

How do we know how good are the models?

## Regression Metrics
(goodness of fit)

### Mean Squared Error

Mean Squared Error(MSE) is the average of the square between the Original Values and the Predicted Values.It gives an idea of how wrong the predictions were.

$$\Large MSE = \frac{1}{N} \sum\limits_{i = 1}^{N} ( {y_i - \hat{y_i}} )^2$$

$$\Large MSE = \Large Average(Actual\ target\ value - Predicted\ target\ value)^2$$

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mse1 = mean_squared_error(y_test, predictions_1)
mse2 = mean_squared_error(y_test, predictions_2)

The squared root will give a better perspective

In [ ]:
print(sqrt(mse1), sqrt(mse2))

In [ ]:
dataset.price.mean()

What does this mean?

![](../../img/reg1.png)

### R Squared

The $R^2$ (or R Squared) metric provides an indication of how well the model captures the variance in the data. It ranges between 0 and 1 for no-variance explained to complete variance captured respectively.

$$(Residual\ Square\ sum)\ \Large RSS = \Large \sum\limits_{i = 1}^{N} (Actual\ target\ value - Predicted\ target\ value)$$

$$(Total\ Square\ sum)\ \Large TSS = \Large \sum\limits_{i = 1}^{N} (Actual\ target\ value - Mean\ target\ value)$$

$$\Large R^2 = \Large (1 - \frac{RSS}{TSS})$$

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score1 = r2_score(y_test, predictions_1)
r2_score2 = r2_score(y_test, predictions_2)

In [ ]:
print(r2_score1, r2_score2)

In [ ]:
plt.bar([1,2], [r2_score1, r2_score2])
plt.xticks([1,2], ['model1','model2'])
plt.ylabel('R2 Score')

The closest to one the better otherwise isn't explaining the variance of the errors.

How to select the most relevant features for prediction?

### Recursive Feature Elimination - Cross Validation

Recursive feature elimination (RFE) is a feature selection method that fits a model and removes the weakest feature (or features) until the specified number of features is reached. Features are ranked by the model’s coef_ or feature_importances_ attributes, and by recursively eliminating a small number of features per loop, RFE attempts to eliminate dependencies and collinearity that may exist in the model.

In [ ]:
from sklearn.feature_selection import RFECV

In [ ]:
lr = LinearRegression()
selector = RFECV(lr,cv=5) # n fold cross validation
selector.fit(X_train, y_train)

In [ ]:
optimized_columns = X_train.columns[selector.support_]
optimized_columns

In [ ]:
selector.ranking_

### Score distribution

In [ ]:
model_scores = []
for repetition in range(1000):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    rmse = sqrt(mean_squared_error(y_test, y_predict))
    model_scores.append(rmse)

In [ ]:
plt.title('Evaluation Score Distribution')
sns.distplot(model_scores, color="g")

### Pros and Cons of Regression Evaluation Metrics
RMSE is the better choice if we only care about model accuracy.  
Any metric such like MAE which doesn’t take the square of the difference is more robust to outliers.  
Any square of error term metric should be more useful when large errors are particularly undesirable.  

## Cross Validation

<img style="float:left;" src="https://cdn-images-1.medium.com/max/1600/1*J2B_bcbd1-s1kpWOu_FZrg.png" width=700 height=300>

## Statmodels

There is always the classical version of regression on a python implementation

In [ ]:
import statsmodels.formula.api as sm

In [ ]:
regressor_OLS = sm.OLS(endog = y, exog=X).fit()

In [ ]:
regressor_OLS.summary()